In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import math

%matplotlib inline

In [4]:
data = pd.read_csv(r'../../data/all_wells.csv')
# data.fillna(data.mean(), inplace=True)
data.head()

,easting,northing,porosity,permeability,Poisson's ratio,Young's Modulus,water saturation,oil saturation,proppant weight (lbs),pump rate (cubic feet/min),name,cumulative production
0,66100.0,22300.0,0.09,0.033,0.332,9440769.483,0.124740,0.875260,260036.414279,275.737593,Tarragon 4-119H,81324.0
1,66199.0,22300.0,0.12,0.057,0.332,9429043.880,0.124979,0.875021,NaN,NaN,Tarragon 4-119H,81324.0
2,66297.0,22300.0,0.11,0.050,0.332,9417413.010,0.125221,0.874779,429740.754787,324.145032,Tarragon 4-119H,81324.0
3,66396.0,22300.0,0.08,0.024,0.332,9405879.454,0.125469,0.874531,NaN,NaN,Tarragon 4-119H,81324.0
4,66495.0,22300.0,0.08,0.031,0.332,9394445.773,0.125720,0.874280,485657.822229,320.868488,Tarragon 4-119H,81324.0


Formula for original oil in place

OOIP = 7758 * length * proppant weight * porosity * 1/proppant density converted to lbs * (1/frac width converted to ft) * (1 acre/43560 feet) * porosity * saturation / fvf

In [3]:
# formula modified to utilize proppant weight, porosity, oil saturation, and well length (acres)
# assumed fvf = 1.6, proppant density 2.65 g/cc, fracture width 0.4 in
def ooip(proppant_weight, porosity, saturation, length, fvf = 1.6):
    return (7758*30/165.43/fvf)*(proppant_weight)*(porosity)*(1-porosity)*(saturation)*(length) / 43560

Alternate approximation for OOIP 

In [4]:
def alt_ooip(cumulative_prod, recovery_rate):
    return 4/3 * 1/recovery_rate * cumulative_prod

Functions generate regressors based on porosity, permeability, poisson's ratio, young's modulus, and oil saturation

In [5]:
def model_length():
    features = data[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation', 'well length (ft)']].dropna()
    X = features[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation']]
    y = features[['well length (ft)']]
    
    model = RandomForestRegressor()
    model.fit(X, np.ravel(y))
    
    return model

In [6]:
def model_frac():
    features = data[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation', 'frac stages']].dropna()
    X = features[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation']]
    y = features[['frac stages']]
    
    model = RandomForestRegressor()
    model.fit(X, np.ravel(y))

    return model

In [7]:
def model_total_proppant():
    features = data[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation', 'total proppant (lbs)']].dropna()
    X = features[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation']]
    y = features[['total proppant (lbs)']]
    
    model = RandomForestRegressor()
    model.fit(X, np.ravel(y))

    return model

In [8]:
def model_total_pump_rate():
    features = data[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation', 'total pump rate (cubic feet/min)']].dropna()
    X = features[['porosity', 'permeability', 'Poisson\'s ratio', 'Young\'s Modulus', 'oil saturation']]
    y = features[['total pump rate (cubic feet/min)']]
    
    model = RandomForestRegressor()
    model.fit(X, np.ravel(y))

    return model 

Establishing regressors

In [9]:
# based on porosity, permeability, poisson's ratio,
# young's modulus, and oil saturation
length_reg = model_length()
frac_reg = model_frac()

In [10]:
total_prop_reg = model_total_proppant()
total_rate_reg = model_total_pump_rate()

Returns list of deliverables given easting and northing

In [11]:
"""
returns a list containing deliverables in the followng order:
    A. easting
    B. northing
    C. length of well
    D. direction
    E. number of frac stages
    F. amount of proppant for each stage
    G. pump rate
    H. original oil in place
    I. recoverable reserves
    J. estimated ultimate recovery
"""
def deliverables(easting, northing, direction, youngs, saturation, poissons, porosity, 
                 permeability, cumulative, low_recovery, high_recovery):
    deliver = []
    
    length = length_reg.predict([[porosity, permeability, poissons, youngs, saturation]])[0]
    frac = frac_reg.predict([[porosity, permeability, poissons, youngs, saturation]])[0]
    total_prop = total_prop_reg.predict([[porosity, permeability, poissons, youngs, saturation]])[0]
    total_rate = total_rate_reg.predict([[porosity, permeability, poissons, youngs, saturation]])[0]
    
    # AB: easting, northing
    deliver.append(easting)
    deliver.append(northing)
    
    # C: length
    deliver.append(length)
    
    # D: direction
    deliver.append(direction)
    
    # E: frac stages
    # rounded up to nearest integer
    deliver.append(math.ceil(frac))
    
    # F: average proppant per stage
    deliver.append(total_prop / frac)
    
    # G: average pump rate per stage
    deliver.append(total_rate / frac)
    
    # H: original oil in place
    # area of well is approximated as length^2
    deliver.append(ooip(total_prop, porosity, saturation, length))
    
    # I: recoverable reserves
    # recoverable reserves == cumulative production
    deliver.append(cumulative)
    
    # J: estimated ultimate recovery
    deliver.append(cumulative)
    
    # alternate calculations for ooip
    deliver.append(alt_ooip(cumulative, low_recovery))
    deliver.append(alt_ooip(cumulative, high_recovery))
    
    return deliver

In [6]:
prospective_wells = pd.read_csv(r'../../data/prospective_wells.csv')
prospective_wells.head()

,easting,northing,porosity,permeability,poissons,youngs,oil,production
0,68449.285714,2414.285714,0.088332,0.039101,0.322992,9.999696e+06,0.852954,116151.000000
1,70373.551020,400.000000,0.088147,0.038545,0.321101,9.729357e+06,0.847988,116151.000000
2,70373.551020,2414.285714,0.088332,0.037889,0.320001,9.756426e+06,0.837453,116151.000000
3,70373.551020,4428.571429,0.088332,0.037889,0.318011,9.776097e+06,0.828860,116151.000000
4,74222.081633,2414.285714,0.084207,0.036734,0.315964,9.246381e+06,0.814396,116011.426667


In [13]:
well_deliverables = []

# top wells from prospective_wells.csv
top_well_indices = [1, 3, 5, 6, 10, 11, 13, 15, 17, 18,]
top_well_directions = ['west', 'west', 'west', 'east', 'west', 'east', 'east', 'west', 'west', 'west',]
# approximated recovery rate based on decay rate of 0.1155
upper_recovery = 0.35
lower_recovery = 0.3

for i in range(len(top_well_indices)):
    row = prospective_wells.iloc[top_well_indices[i]]
    well_deliverables.append(deliverables(row['easting'], row['northing'], top_well_directions[i],
                                         row['youngs'], row['oil'], row['poissons'], 
                                         row['porosity'], row['permeability'], row['production'],
                                         upper_recovery, lower_recovery))

In [14]:
# creating DataFrame from list top 10 wells
col_names = ['easting (ft)', 'northing (ft)', 'length of well (ft)', 
             'direction', 'frac stages', 'average proppant per frac stage (lb)', 
             'average pump rate per frac stage (cubic feet/min)', 'original oil in place (bbl)',
            'recoverable reserves (bbl)', 'estimated ultimate recovery (bbl)',
            '(alternate) low original oil in place (bbl)', '(alternate) high original oil in place (bbl)']

all_deliverables = pd.DataFrame(well_deliverables, columns = col_names)

Total ooip / recovery factor

In [15]:
total_ooip = all_deliverables['original oil in place (bbl)'].sum()
total_rr = all_deliverables['recoverable reserves (bbl)'].sum()
total_eur = all_deliverables['estimated ultimate recovery (bbl)'].sum()

# recovery factor is the recoverable reserves / original oil in place
recovery_factor = total_rr / total_ooip

totals = pd.DataFrame([[total_ooip, total_rr, total_eur, recovery_factor]], columns = ['total original oil in place (bbl)', 
                                                                      'total recoverable reserves (bbl)', 
                                                                      'total estimated ultimate recovery (bbl)',
                                                                        'recovery factor'])
totals

,total original oil in place (bbl),total recoverable reserves (bbl),total estimated ultimate recovery (bbl),recovery factor
0,4.369650e+09,1.138053e+06,1.138053e+06,0.00026


Alternate ooip / recovery factor

In [16]:
# alternate low / high ooip and recovery factor
low_alt_ooip = all_deliverables['(alternate) low original oil in place (bbl)'].sum()
low_alt_ooip_recovery = total_rr / low_alt_ooip

high_alt_ooip = all_deliverables['(alternate) high original oil in place (bbl)'].sum()
high_alt_ooip_recovery = total_rr / high_alt_ooip

alt_totals = pd.DataFrame([[low_alt_ooip, low_alt_ooip_recovery, high_alt_ooip, high_alt_ooip_recovery]], columns = 
                          ['total (alternate) low original oil in place (bbl)',
                          'alternate low ooip recovery factor',
                          'total (alternate) high original oil in place (bbl)',
                          'alternate high ooip recovery factor'])
alt_totals

,total (alternate) low original oil in place (bbl),alternate low ooip recovery factor,total (alternate) high original oil in place (bbl),alternate high ooip recovery factor
0,4.335442e+06,0.2625,5.058015e+06,0.225


In [17]:
# rounding values to nearest integer after calculations
all_deliverables = all_deliverables.round(0)
all_deliverables

,easting (ft),northing (ft),length of well (ft),direction,frac stages,average proppant per frac stage (lb),average pump rate per frac stage (cubic feet/min),original oil in place (bbl),recoverable reserves (bbl),estimated ultimate recovery (bbl),(alternate) low original oil in place (bbl),(alternate) high original oil in place (bbl)
0,70374.0,400.0,7962.0,west,50,829209.0,305.0,454188313.0,116151.0,116151.0,442480.0,516227.0
1,70374.0,4429.0,7785.0,west,50,840128.0,303.0,437972248.0,116151.0,116151.0,442480.0,516227.0
2,70374.0,12486.0,8008.0,west,50,803007.0,304.0,427455493.0,115765.0,115765.0,441009.0,514510.0
3,72298.0,12486.0,7809.0,east,50,835087.0,304.0,412732732.0,115346.0,115346.0,439414.0,512649.0
4,68449.0,8457.0,7735.0,west,50,829150.0,301.0,437030260.0,113748.0,113748.0,433324.0,505545.0
5,68449.0,10471.0,7734.0,east,50,827828.0,301.0,434607239.0,113748.0,113748.0,433324.0,505545.0
6,51131.0,14500.0,8650.0,east,50,671381.0,302.0,434260741.0,112454.0,112454.0,428396.0,499796.0
7,53055.0,12486.0,8647.0,west,50,671381.0,302.0,436661174.0,112411.0,112411.0,428233.0,499605.0
8,68449.0,6443.0,7703.0,west,50,829150.0,303.0,436293617.0,111450.0,111450.0,424571.0,495333.0
9,49207.0,14500.0,8648.0,west,50,671381.0,302.0,458447826.0,110830.0,110830.0,422210.0,492579.0
